In [19]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [21]:
tf.keras.backend.clear_session()

In [22]:
def add_cos_monthly(df, column_name):
    values = [np.cos(2 * np.pi * (i % 12) / 12) for i in range(len(df))]
    df[column_name] = values
    return df

In [23]:
subject = 'Ceará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [24]:
data = pd.read_csv('2003_mo_model_input_CE.csv')
# data = add_cos_monthly(data, "Sazonalidade")
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Ceará - Consumo de Cimento (t)
0,2003-1,8.274459,0.301376,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.684040,6.965352e+07,4.525724e+06,7.532751,6.293821e+07,107.673000,81.265
1,2003-2,8.268512,0.301393,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.684476,6.974354e+07,4.530359e+06,7.535808,6.297178e+07,107.237962,65.200
2,2003-3,8.262564,0.301309,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.684912,6.983356e+07,4.534994e+06,7.538865,6.300536e+07,108.742184,55.165
3,2003-4,8.256617,0.300917,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.685347,6.992358e+07,4.539629e+06,7.541921,6.303893e+07,109.391112,53.928
4,2003-5,8.250670,0.300417,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.685783,7.001361e+07,4.544265e+06,7.544978,6.307250e+07,111.602457,65.334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.535686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.122997,186.918
236,2022-9,NaN,0.533990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.220173,179.828
237,2022-10,NaN,0.530449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237.879214,174.940
238,2022-11,NaN,0.526253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.799325,182.597


In [25]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [26]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      75.575
1      53.065
2      67.755
3      64.858
4      72.377
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Ceará - Consumo de Cimento (t), Length: 240, dtype: float64

In [27]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.423186,-0.021469,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.409954,1.190773,-0.838130,0.874129,1.017258,1.523635
158,1.399318,-0.031468,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.414132,1.193287,-0.881990,0.871719,1.015919,1.541381
159,1.375451,-0.044253,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.418310,1.195800,-0.925849,0.869309,1.014581,1.523536
160,1.351584,-0.046806,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422488,1.198313,-0.969709,0.866899,1.013242,1.507028


In [28]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       75.575
1       53.065
2       67.755
3       64.858
4       72.377
        ...   
157    134.739
158    151.819
159    124.858
160    162.684
161    169.251
Name: Ceará - Consumo de Cimento (t), Length: 162, dtype: float64

In [29]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [30]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [31]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
126,0.941464,1.223482,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.365676,0.843485,0.634116,1.037548,1.010785,0.074200
127,0.969984,1.232594,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.371209,0.863067,0.604648,1.047639,1.024388,0.100668
128,0.998504,1.233295,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.376742,0.882650,0.575180,1.057730,1.037991,0.116036
129,1.027024,1.227338,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.382275,0.902233,0.545712,1.067821,1.051594,0.134124
130,1.055544,1.232230,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.387808,0.921816,0.516244,1.077913,1.065197,0.158476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [32]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [33]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [66]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(360, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(72, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        batch_size=36,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [67]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [68]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2241032896, 697336410, 1315391235, 3093478040, 3524950027]


Step: 0 ___________________________________________
val_loss: 472.1915588378906
winner_seed: 2241032896


Step: 1 ___________________________________________
val_loss: 226.7750701904297
winner_seed: 697336410


Step: 2 ___________________________________________
val_loss: 311.8902587890625


Step: 3 ___________________________________________
val_loss: 302.3897399902344


Step: 4 ___________________________________________
val_loss: 397.2163391113281


final_seed: 697336410


In [70]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 140ms/step - loss: 25761.7129 - val_loss: 31231.1250
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 25597.8477 - val_loss: 31136.8027
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 25434.3301 - val_loss: 31017.9414
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 25256.8008 - val_loss: 30888.9668
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 25064.1934 - val_loss: 30736.3125
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 24855.3887 - val_loss: 30555.6172
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 24628.4863 - val_loss: 30332.1055
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 24394.3047 - val_loss: 30083.3555
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 24147.0625 - val_loss: 29829.2988
Epoch 10/10000
4/4 [=======

4/4 [==============================] - 0s 10ms/step - loss: 16159.2109 - val_loss: 21221.1152
Epoch 77/10000
4/4 [==============================] - 0s 10ms/step - loss: 16089.0654 - val_loss: 21139.1738
Epoch 78/10000
4/4 [==============================] - 0s 10ms/step - loss: 16018.5117 - val_loss: 21058.7363
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 15948.6055 - val_loss: 20979.9062
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 15879.3115 - val_loss: 20900.6250
Epoch 81/10000
4/4 [==============================] - 0s 12ms/step - loss: 15810.1201 - val_loss: 20820.8320
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 15740.6182 - val_loss: 20741.2480
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 15671.6504 - val_loss: 20660.8125
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 15601.5596 - val_loss: 20580.8535
Epoch 85/10000
4/4 [==============

Epoch 151/10000
4/4 [==============================] - 0s 11ms/step - loss: 11651.0850 - val_loss: 15892.0947
Epoch 152/10000
4/4 [==============================] - 0s 10ms/step - loss: 11599.4268 - val_loss: 15826.5195
Epoch 153/10000
4/4 [==============================] - 0s 11ms/step - loss: 11547.8770 - val_loss: 15763.2021
Epoch 154/10000
4/4 [==============================] - 0s 10ms/step - loss: 11497.3926 - val_loss: 15700.1465
Epoch 155/10000
4/4 [==============================] - 0s 11ms/step - loss: 11446.9756 - val_loss: 15636.1455
Epoch 156/10000
4/4 [==============================] - 0s 10ms/step - loss: 11397.3027 - val_loss: 15572.5459
Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 11346.5508 - val_loss: 15511.5898
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 11298.3906 - val_loss: 15451.1055
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 11249.8105 - val_loss: 15391.0244
Epoch 160/

Epoch 226/10000
4/4 [==============================] - 0s 11ms/step - loss: 8346.3574 - val_loss: 11688.9883
Epoch 227/10000
4/4 [==============================] - 0s 11ms/step - loss: 8309.2852 - val_loss: 11640.2324
Epoch 228/10000
4/4 [==============================] - 0s 10ms/step - loss: 8273.0576 - val_loss: 11590.7100
Epoch 229/10000
4/4 [==============================] - 0s 10ms/step - loss: 8236.2314 - val_loss: 11541.7930
Epoch 230/10000
4/4 [==============================] - 0s 9ms/step - loss: 8199.9062 - val_loss: 11493.7275
Epoch 231/10000
4/4 [==============================] - 0s 10ms/step - loss: 8163.5581 - val_loss: 11446.6162
Epoch 232/10000
4/4 [==============================] - 0s 10ms/step - loss: 8128.2070 - val_loss: 11399.9385
Epoch 233/10000
4/4 [==============================] - 0s 10ms/step - loss: 8092.8931 - val_loss: 11352.9082
Epoch 234/10000
4/4 [==============================] - 0s 12ms/step - loss: 8056.7778 - val_loss: 11306.5625
Epoch 235/10000
4/4 

4/4 [==============================] - 0s 11ms/step - loss: 6021.4409 - val_loss: 8539.1699
Epoch 302/10000
4/4 [==============================] - 0s 11ms/step - loss: 5994.7002 - val_loss: 8502.5781
Epoch 303/10000
4/4 [==============================] - 0s 10ms/step - loss: 5969.9556 - val_loss: 8464.9199
Epoch 304/10000
4/4 [==============================] - 0s 10ms/step - loss: 5943.5220 - val_loss: 8427.3164
Epoch 305/10000
4/4 [==============================] - 0s 11ms/step - loss: 5916.5757 - val_loss: 8388.7979
Epoch 306/10000
4/4 [==============================] - 0s 11ms/step - loss: 5889.4766 - val_loss: 8350.9258
Epoch 307/10000
4/4 [==============================] - 0s 11ms/step - loss: 5863.2578 - val_loss: 8314.7881
Epoch 308/10000
4/4 [==============================] - 0s 12ms/step - loss: 5837.1079 - val_loss: 8279.8994
Epoch 309/10000
4/4 [==============================] - 0s 11ms/step - loss: 5812.6333 - val_loss: 8244.7598
Epoch 310/10000
4/4 [=======================

...

4/4 [==============================] - 0s 10ms/step - loss: 94.5471 - val_loss: 319.4150
Epoch 1984/10000
4/4 [==============================] - 0s 10ms/step - loss: 94.0027 - val_loss: 323.2102
Epoch 1985/10000
4/4 [==============================] - 0s 10ms/step - loss: 91.3021 - val_loss: 321.2205
Epoch 1986/10000
4/4 [==============================] - 0s 10ms/step - loss: 87.3802 - val_loss: 320.9783
Epoch 1987/10000
4/4 [==============================] - 0s 9ms/step - loss: 86.7518 - val_loss: 326.1722
Epoch 1988/10000
4/4 [==============================] - 0s 10ms/step - loss: 87.2681 - val_loss: 330.4582
Epoch 1989/10000
4/4 [==============================] - 0s 9ms/step - loss: 85.9693 - val_loss: 326.2484
Epoch 1990/10000
4/4 [==============================] - 0s 10ms/step - loss: 89.8771 - val_loss: 316.2271
Epoch 1991/10000
4/4 [==============================] - 0s 10ms/step - loss: 88.9859 - val_loss: 314.7399
Epoch 1992/10000
4/4 [==============================] - 0s 10ms/s

4/4 [==============================] - 0s 10ms/step - loss: 83.8778 - val_loss: 717.4747
Epoch 2061/10000
4/4 [==============================] - 0s 10ms/step - loss: 83.2521 - val_loss: 719.7190
Epoch 2062/10000
4/4 [==============================] - 0s 10ms/step - loss: 81.0335 - val_loss: 724.4044
Epoch 2063/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.3654 - val_loss: 735.0938
Epoch 2064/10000
4/4 [==============================] - 0s 10ms/step - loss: 82.1553 - val_loss: 731.7584
Epoch 2065/10000
4/4 [==============================] - 0s 9ms/step - loss: 81.0102 - val_loss: 719.8930
Epoch 2066/10000
4/4 [==============================] - 0s 10ms/step - loss: 80.1511 - val_loss: 723.4659
Epoch 2067/10000
4/4 [==============================] - 0s 9ms/step - loss: 78.8271 - val_loss: 749.3646
Epoch 2068/10000
4/4 [==============================] - 0s 10ms/step - loss: 77.7105 - val_loss: 770.9547
Epoch 2069/10000
4/4 [==============================] - 0s 10ms/s

4/4 [==============================] - 0s 10ms/step - loss: 78.6863 - val_loss: 739.9291
Epoch 2138/10000
4/4 [==============================] - 0s 10ms/step - loss: 78.9303 - val_loss: 747.0270
Epoch 2139/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.0563 - val_loss: 748.5798
Epoch 2140/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.3297 - val_loss: 748.2169
Epoch 2141/10000
4/4 [==============================] - 0s 11ms/step - loss: 77.0228 - val_loss: 748.4561
Epoch 2142/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.1123 - val_loss: 748.5771
Epoch 2143/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.3367 - val_loss: 750.4418
Epoch 2144/10000
4/4 [==============================] - 0s 11ms/step - loss: 77.7283 - val_loss: 751.0704
Epoch 2145/10000
4/4 [==============================] - 0s 11ms/step - loss: 75.7564 - val_loss: 746.9342
Epoch 2146/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 10ms/step - loss: 103.5060 - val_loss: 730.5144
Epoch 2215/10000
4/4 [==============================] - 0s 10ms/step - loss: 102.0021 - val_loss: 725.4133
Epoch 2216/10000
4/4 [==============================] - 0s 10ms/step - loss: 100.8014 - val_loss: 722.1357
Epoch 2217/10000
4/4 [==============================] - 0s 10ms/step - loss: 97.1216 - val_loss: 717.0432
Epoch 2218/10000
4/4 [==============================] - 0s 10ms/step - loss: 101.3791 - val_loss: 718.3278
Epoch 2219/10000
4/4 [==============================] - 0s 9ms/step - loss: 105.3195 - val_loss: 726.0723
Epoch 2220/10000
4/4 [==============================] - 0s 11ms/step - loss: 110.0435 - val_loss: 723.6439
Epoch 2221/10000
4/4 [==============================] - 0s 10ms/step - loss: 130.9622 - val_loss: 714.1019
Epoch 2222/10000
4/4 [==============================] - 0s 10ms/step - loss: 133.8668 - val_loss: 700.7617
Epoch 2223/10000
4/4 [==============================] - 

4/4 [==============================] - 0s 12ms/step - loss: 72.9426 - val_loss: 778.8738
Epoch 2292/10000
4/4 [==============================] - 0s 11ms/step - loss: 80.2562 - val_loss: 779.9657
Epoch 2293/10000
4/4 [==============================] - 0s 10ms/step - loss: 67.0645 - val_loss: 778.2227
Epoch 2294/10000
4/4 [==============================] - 0s 10ms/step - loss: 69.7626 - val_loss: 778.3155
Epoch 2295/10000
4/4 [==============================] - 0s 10ms/step - loss: 71.8328 - val_loss: 783.0302
Epoch 2296/10000
4/4 [==============================] - 0s 10ms/step - loss: 67.5047 - val_loss: 785.6962
Epoch 2297/10000
4/4 [==============================] - 0s 10ms/step - loss: 66.5663 - val_loss: 789.2836
Epoch 2298/10000
4/4 [==============================] - 0s 11ms/step - loss: 66.6642 - val_loss: 784.4942
Epoch 2299/10000
4/4 [==============================] - 0s 10ms/step - loss: 70.5179 - val_loss: 781.7041
Epoch 2300/10000
4/4 [==============================] - 0s 11ms

In [71]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [72]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,161.276474,157.797745,153.00145,144.995926,129.771606,114.384872,111.197685,110.250816,110.393295,110.172112,109.392525,107.809639,104.364693,100.681328,100.048515,100.539474,101.671852,104.094566,102.162109,104.396362,105.368889,104.784966,104.581772,104.338783,103.608345,102.981758,100.707237,98.05497,95.122452,92.293518
Target,179.243,194.134,186.948,194.907,179.505,173.278,160.261,113.987,114.289,117.313,110.502,147.996,152.386,177.259,155.913,172.41,179.419,157.165,151.601,129.537,105.236,110.835,127.925,127.138,157.128,164.44,140.298,168.407,163.615,155.07
Error,17.966522,36.336258,33.946548,49.911072,49.733398,58.893127,49.063316,3.736183,3.895706,7.140892,1.109474,40.186363,48.021309,76.577675,55.864479,71.870529,77.747154,53.070427,49.438889,25.14064,0.132889,6.050034,23.343231,22.799217,53.519661,61.458244,39.590767,70.352028,68.492554,62.776489


In [73]:
display(mae)
display(mape)

40.605503

0.25087824

In [74]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [75]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1520.4443]] - Target[1872.363]| =  Error: [[351.9187]]; MAPE:[[0.1879543]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1237.0333]] - Target[1746.8239999999998]| =  Error: [[509.79065]]; MAPE:[[0.2918386]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[592.76825]] - Target[948.9580000000001]| =  Error: [[356.18976]]; MAPE:[[0.37534827]]


[array([[351.9187]], dtype=float32),
 array([[509.79065]], dtype=float32),
 array([[356.18976]], dtype=float32)]

405.9664

0.28504705